In [1]:
from data_pipeline import PipelineGenerator

import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf

%matplotlib inline

In [2]:
from models import ModelFactory

In [3]:
from typing import List, Callable, Optional

In [4]:
metadata_file = "../data/final_dataset_test_balanced-shuffled.csv"
wellington_images_dir = "../../wellington_data/images-resized-224/" # Ensure the slash at the end.

pipeline_gen = PipelineGenerator(metadata_file,
                                 wellington_images_dir,
                                 is_training=False,
                                 sequence_image_count=3,
                                 label_name="has_animal",
                                 mode="mode_flat_all")

In [5]:
test_dataset_raw = pipeline_gen.get_pipeline()
test_dataset_batches = test_dataset_raw.batch(32).prefetch(tf.data.experimental.AUTOTUNE)
num_test_images = pipeline_gen.get_size()
print("There are %s test images." % num_test_images)

There are 13800 test images.


In [6]:
for batch in test_dataset_batches:
    print("Loaded first batch to 'batch'.")
    break

Loaded first batch to 'batch'.


In [7]:
test_images, test_labels = batch

In [8]:
test_labels

<tf.Tensor: shape=(32,), dtype=int64, numpy=
array([1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0])>

In [9]:
test_images

<tf.Tensor: shape=(32, 224, 224, 3), dtype=float32, numpy=
array([[[[0.227451  , 0.227451  , 0.227451  ],
         [0.3647059 , 0.3647059 , 0.3647059 ],
         [0.41960788, 0.41960788, 0.41960788],
         ...,
         [0.24313727, 0.24313727, 0.24313727],
         [0.14901961, 0.14901961, 0.14901961],
         [0.13725491, 0.13725491, 0.13725491]],

        [[0.21568629, 0.21568629, 0.21568629],
         [0.29803923, 0.29803923, 0.29803923],
         [0.38431376, 0.38431376, 0.38431376],
         ...,
         [0.18431373, 0.18431373, 0.18431373],
         [0.14117648, 0.14117648, 0.14117648],
         [0.11764707, 0.11764707, 0.11764707]],

        [[0.1764706 , 0.1764706 , 0.1764706 ],
         [0.24705884, 0.24705884, 0.24705884],
         [0.3372549 , 0.3372549 , 0.3372549 ],
         ...,
         [0.13725491, 0.13725491, 0.13725491],
         [0.12941177, 0.12941177, 0.12941177],
         [0.12156864, 0.12156864, 0.12156864]],

        ...,

        [[0.87843144, 0.5137255 ,

### Loading the model.

In [10]:
model_arch_name = "vgg16_pretrained_imagenet" # "vgg16_batchnorm"
trained_model_weights = "../trained_models/baseline_4/best_model_dir-acc.ckpt/variables/variables" # "../trained_models/baseline_3/best_model_dir-acc.ckpt/variables/variables"

In [11]:
!ls ../trained_models/baseline_3/best_model_dir-acc.ckpt/variables/

variables.data-00000-of-00002  variables.data-00001-of-00002  variables.index


In [12]:
model_factory = ModelFactory()
model = model_factory.get_model(model_arch_name,
                                (224, 224, 3),
                                is_training=False,
                                num_classes=1,
                                learning_rate=0.001)

Available model architectures are: dict_keys(['vgg16_batchnorm', 'vgg16_pretrained_imagenet', 'cnn_lstm'])


In [13]:
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

32/32 - 4s - loss: 0.7160 - accuracy: 0.4375
Untrained model, accuracy: 43.75%


In [14]:
model.load_weights(trained_model_weights)

In [15]:
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Trained model, accuracy: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 1.0203 - accuracy: 0.5000
Trained model, accuracy: 50.00%


In [16]:
preds = model.predict(test_images)

In [17]:
preds_1 = model.predict(test_images)

In [18]:
preds_2 = model.predict(test_images)

In [19]:
preds_3 = model.predict(test_images)

In [20]:
np.allclose(preds_1, preds_2), np.allclose(preds_2, preds_3), np.allclose(preds_1, preds_3)

(True, True, True)

In [21]:
model_pass_preds = model(test_images)

In [22]:
model_pass_preds_1 = model(test_images, training=False)

In [23]:
model_pass_preds_2 = model(test_images, training=True)

In [24]:
np.allclose(preds, model_pass_preds), np.allclose(preds, model_pass_preds_1), np.allclose(preds, model_pass_preds_2)

(True, True, True)

In [25]:
preds_labels = list(zip(preds[:, 0], test_labels.numpy()))
preds_labels

[(0.97622436, 1),
 (0.9776285, 1),
 (0.97648424, 1),
 (0.7638976, 0),
 (0.75954866, 0),
 (0.76894945, 0),
 (0.71649927, 0),
 (0.0624763, 0),
 (0.06282469, 0),
 (0.9050708, 0),
 (0.9046163, 0),
 (0.9017011, 0),
 (0.4425685, 0),
 (0.45698246, 0),
 (0.45727015, 0),
 (0.8741178, 1),
 (0.8732961, 1),
 (0.8657045, 1),
 (0.69335294, 1),
 (0.70069164, 1),
 (0.70188004, 1),
 (0.7720988, 0),
 (0.7750649, 0),
 (0.7695096, 0),
 (0.79775566, 0),
 (0.7992698, 0),
 (0.8025328, 0),
 (0.88109577, 0),
 (0.8756083, 0),
 (0.881826, 0),
 (0.3703212, 0),
 (0.36830407, 0)]

# Do not load the model directly as the batchnorm layers will also be active.

In [28]:
direct_model = tf.keras.models.load_model("../trained_models/baseline_3/best_model_dir-acc.ckpt")
direct_preds = direct_model.predict(test_images)
direct_preds_1 = direct_model.predict(test_images)
direct_preds_2 = direct_model.predict(test_images)

np.allclose(direct_preds, direct_preds_1), np.allclose(direct_preds, direct_preds_2)

(False, False)

In [29]:
direct_model = tf.keras.models.load_model("../trained_models/baseline_3/best_model_dir-acc.ckpt")
direct_preds_model_pass_preds = direct_model(test_images)
direct_preds_model_pass_preds_1 = direct_model(test_images, training=False)
direct_preds_model_pass_preds_2 = direct_model(test_images, training=True)

np.allclose(direct_preds_model_pass_preds, direct_preds_model_pass_preds_1), np.allclose(direct_preds_model_pass_preds, direct_preds_model_pass_preds_2)

(False, False)

In [33]:
# Trying by setting the learning phase.
tf.keras.backend.set_learning_phase(0)
direct_model = tf.keras.models.load_model("../trained_models/baseline_3/best_model_dir-acc.ckpt")
direct_preds = direct_model.predict(test_images)
direct_preds_1 = direct_model.predict(test_images)
direct_preds_2 = direct_model.predict(test_images)

np.allclose(direct_preds, direct_preds_1), np.allclose(direct_preds, direct_preds_2)

(False, False)

### We are able to get consistent prediction results!

In [45]:
# Taken from n2cholas, from link: https://github.com/tensorflow/tensorflow/issues/33478.
class LayerWithHooks(tf.keras.layers.Layer):
    def __init__(self, layer: tf.keras.layers.Layer, hooks: List[Callable[[tf.Tensor, tf.Tensor], Optional[tf.Tensor]]] = None):
        super().__init__()
        self._layer = layer
        self._hooks = hooks or []
  
    def call(self, input: tf.Tensor) -> tf.Tensor:
        output = self._layer(input)
        for hook in self._hooks:
            hook_result = hook(input, output)
            if hook_result is not None:
                output = hook_result
        return output
  
    def register_hook(self, hook: Callable[[tf.Tensor, tf.Tensor], Optional[tf.Tensor]]) -> None:
        self._hooks.append(hook)


def get_call_fn(layer: tf.keras.layers.Layer) -> Callable[[tf.Tensor], tf.Tensor]:
    old_call_fn = layer.call
    def call(input: tf.Tensor) -> tf.Tensor:
        output = old_call_fn(input)
        for hook in layer._hooks:
            hook_result = hook(input, output)
            if hook_result is not None:
                output = hook_result
        return output
    return call


class InputOutputSaver:
    def __call__(self, input: tf.Tensor, output: tf.Tensor) -> None:
        self.input = input
        self.output = output


def get_layer_outputs(model, required_layers, input_batch):
    """
    Extract the outputs of intermediate layers provided in required_layers from the model,
    after running the input_batch through the model.
    Returns a dictionary of 'savers'. Access a particular layer output as followed:
        savers[layer_name].output

    """
    # Register hooks and savers.
    savers = {}
    for layer_name in required_layers:
        layer = model.get_layer(layer_name)
        layer._hooks = []
        layer.call = get_call_fn(layer)
        layer.register_hook = lambda hook: layer._hooks.append(hook)
        saver = InputOutputSaver()
        layer.register_hook(saver)
        savers[layer_name] = saver

    model_outputs = model(input_batch)
    return savers


In [46]:
model.summary()

Model: "vgg16_batchnorm"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv1-BN (BatchNormal (None, 224, 224, 64)      256       
_________________________________________________________________
block1_conv1-Relu (ReLU)     (None, 224, 224, 64)      0         
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_conv2-BN (BatchNormal (None, 224, 224, 64)      256       
_________________________________________________________________
block1_conv2-Relu (ReLU)     (None, 224, 224, 64)  

In [48]:
required_layers = ["block1_conv1-Relu", "block3_conv3-Relu", "block5_conv3-Relu"]
savers = get_layer_outputs(model, required_layers, test_images)

In [49]:
savers["block5_conv3-Relu"].output.shape

TensorShape([32, 14, 14, 512])

In [41]:
print(model(test_images))
print(savers[layer_name].output)

tf.Tensor(
[[0.8653604 ]
 [0.8644319 ]
 [0.8637273 ]
 [0.7385264 ]
 [0.7346206 ]
 [0.7335631 ]
 [0.7210164 ]
 [0.30007064]
 [0.24799033]
 [0.86783844]
 [0.8681652 ]
 [0.8682599 ]
 [0.7816132 ]
 [0.781556  ]
 [0.78165483]
 [0.896185  ]
 [0.896271  ]
 [0.8961991 ]
 [0.755492  ]
 [0.7543403 ]
 [0.7553325 ]
 [0.8690139 ]
 [0.8774575 ]
 [0.8793199 ]
 [0.8509181 ]
 [0.8535819 ]
 [0.85290736]
 [0.8136687 ]
 [0.8118903 ]
 [0.81214046]
 [0.7432765 ]
 [0.7423498 ]], shape=(32, 1), dtype=float32)
tf.Tensor(
[[[[0.00000000e+00 1.01214588e+00 0.00000000e+00 ... 1.19935167e+00
    1.43341196e+00 1.16697991e+00]
   [0.00000000e+00 4.42889601e-01 0.00000000e+00 ... 8.17812562e-01
    1.03941059e+00 8.17280471e-01]
   [0.00000000e+00 2.30979785e-01 0.00000000e+00 ... 6.76411092e-01
    9.16516423e-01 6.97949648e-01]
   ...
   [0.00000000e+00 9.28480744e-01 0.00000000e+00 ... 1.01047003e+00
    1.44530821e+00 1.04737210e+00]
   [0.00000000e+00 1.19649446e+00 0.00000000e+00 ... 1.19119930e+00
    1.64296

In [42]:
layer_out = savers[layer_name].output

In [43]:
layer_out.shape

TensorShape([32, 224, 224, 64])

In [44]:
type(layer_out)

tensorflow.python.framework.ops.EagerTensor

### Testing with sequence mode of data pipeline.

In [51]:
pipeline_gen = PipelineGenerator(metadata_file,
                                 wellington_images_dir,
                                 is_training=False,
                                 sequence_image_count=3,
                                 label_name="has_animal",
                                 mode="mode_sequence")

test_dataset_raw = pipeline_gen.get_pipeline()
test_dataset_batches = test_dataset_raw.batch(32).prefetch(tf.data.experimental.AUTOTUNE)
num_test_sequences = pipeline_gen.get_size()
print("There are %s test sequences." % num_test_sequences)

There are 4600 test sequences.


In [52]:
for batch in test_dataset_batches:
    print("Loaded first batch to 'batch'.")
    break

Loaded first batch to 'batch'.


In [53]:
test_sequences, labels = batch

In [54]:
test_sequences.shape

TensorShape([32, 3, 224, 224, 3])

In [55]:
a = test_sequences[:, 0, :, :, :]
a.shape

TensorShape([32, 224, 224, 3])

In [56]:
a = [1,2,3]
a.extend([4,5,6])
a

[1, 2, 3, 4, 5, 6]

In [58]:
[[]] * 3

[[], [], []]

In [59]:
import pickle

In [60]:
pickle.dump?

In [70]:
all_pred_labels = [preds_labels, preds_labels, preds_labels]

In [73]:
individual_preds_0 = [x[0] for x in preds_labels]

In [90]:
individual_labels_0 = [x[1] for x in preds_labels]

In [74]:
individual_preds_0

[0.8653604,
 0.864432,
 0.86372733,
 0.7385264,
 0.73462063,
 0.733563,
 0.72101635,
 0.30007064,
 0.2479903,
 0.86783844,
 0.8681652,
 0.86826,
 0.7816132,
 0.781556,
 0.78165483,
 0.896185,
 0.896271,
 0.8961991,
 0.755492,
 0.75434023,
 0.7553325,
 0.86901385,
 0.8774575,
 0.8793199,
 0.8509181,
 0.8535819,
 0.85290736,
 0.8136687,
 0.8118903,
 0.81214046,
 0.7432765,
 0.7423498]

In [77]:
a = np.vstack((individual_preds_0, individual_preds_0, individual_preds_0))

In [78]:
a.shape

(3, 32)

In [81]:
b = np.max(a, axis=0)

In [82]:
b

array([0.8653604 , 0.864432  , 0.86372733, 0.7385264 , 0.73462063,
       0.733563  , 0.72101635, 0.30007064, 0.2479903 , 0.86783844,
       0.8681652 , 0.86826   , 0.7816132 , 0.781556  , 0.78165483,
       0.896185  , 0.896271  , 0.8961991 , 0.755492  , 0.75434023,
       0.7553325 , 0.86901385, 0.8774575 , 0.8793199 , 0.8509181 ,
       0.8535819 , 0.85290736, 0.8136687 , 0.8118903 , 0.81214046,
       0.7432765 , 0.7423498 ], dtype=float32)

In [84]:
np.allclose?

In [86]:
np.allclose(individual_preds_0, individual_preds_0)

True

In [88]:
np.allclose(individual_preds_0, individual_preds_0)

True

In [89]:
b.shape

(32,)

In [91]:
individual_labels_0

AttributeError: 'list' object has no attribute 'shape'

In [92]:
list(zip(b, individual_labels_0))

[(0.8653604, 1),
 (0.864432, 1),
 (0.86372733, 1),
 (0.7385264, 0),
 (0.73462063, 0),
 (0.733563, 0),
 (0.72101635, 0),
 (0.30007064, 0),
 (0.2479903, 0),
 (0.86783844, 0),
 (0.8681652, 0),
 (0.86826, 0),
 (0.7816132, 0),
 (0.781556, 0),
 (0.78165483, 0),
 (0.896185, 1),
 (0.896271, 1),
 (0.8961991, 1),
 (0.755492, 1),
 (0.75434023, 1),
 (0.7553325, 1),
 (0.86901385, 0),
 (0.8774575, 0),
 (0.8793199, 0),
 (0.8509181, 0),
 (0.8535819, 0),
 (0.85290736, 0),
 (0.8136687, 0),
 (0.8118903, 0),
 (0.81214046, 0),
 (0.7432765, 0),
 (0.7423498, 0)]